In [1]:
%cd ..

c:\Users\youse\Desktop\Research\Apperception\apperception


In [2]:
from apperception.database import database

In [ ]:
############ ROAD DIRECTION ############

In [4]:
### Add Road Direction Attributes to Tables ###
query = """ALTER TABLE Item_General_Trajectory
           ADD roadDirection tfloat;

           ALTER TABLE Cameras
           ADD roadDirection numeric;
        """
database._execute_query(query=query)

In [5]:
### Update Road Direction For Camera ###
query = """UPDATE Cameras
           SET roadDirection = roadDirection(egoTranslation, egoHeading)
           WHERE roadDirection IS NULL;
        """
database._execute_query(query=query)

In [7]:
### Create Function For roadDirection With Traj ###
query = """ 
        DROP FUNCTION IF EXISTS _roadDirectionAttr(trajectory tgeompoint, headings tfloat);
        CREATE OR REPLACE FUNCTION _roadDirectionAttr(trajectory tgeompoint, headings tfloat) RETURNS tfloat AS
        $BODY$
        declare trajString text;
        declare trajPointString text;
        declare trajPoint geometry;
        declare trajTime timestamptz;
        declare result tfloat;
        declare currentRoadDir float;
        declare currentRoadDirTime tfloat;
        BEGIN
        trajString := asText(trajectory);
        trajString := substr(trajString, 2, length(trajString) - 2);
        result := tfloat '';
        FOREACH trajPointString IN ARRAY STRING_TO_ARRAY(trajString, ',')
        LOOP
                raise notice 'trajPoint: %', STRING_TO_ARRAY(trajPointString, '@');
                trajPoint := ST_GeomFromText((STRING_TO_ARRAY(trajPointString, '@'))[0]);
                trajTime :=  (STRING_TO_ARRAY(trajPointString, '@'))[2];

                currentRoadDir := roadDirection(trajPoint, valueAtTimestamp(headings, trajTime));
                currentRoadDirTime := currentRoadDir + '@' + trajTime;
                result := appendInstant(result, currentRoadDirTime);
        END LOOP;
        END
        $BODY$
        LANGUAGE 'plpgsql' ;

        SELECT _roadDirectionAttr(tgeompoint  '[Point(1 1)@2001-01-01, Point(2 2)@2001-01-03, POINT(3 3)@2001-01-05]', tfloat  '[1@2001-01-01, 2@2001-01-03, 3@2001-01-05]')
        """
database._execute_query(query=query)
# SELECT STRING_TO_ARRAY(asText(tgeompoint  '[Point(1 1)@2001-01-01, Point(2 2)@2001-01-03, POINT(3 3)@2001-01-05]'), ',');


InvalidTextRepresentation: Could not parse element value
LINE 1: result := tfloat ''
                         ^
QUERY:  result := tfloat ''
CONTEXT:  PL/pgSQL function _roaddirectionattr(tgeompoint,tfloat) line 12 at assignment


In [ ]:
############ MIN AND MAX TIMESTAMPS ############

In [ ]:
### Add minTimestamp Attributes to Trajectory Table ###
query = """ALTER TABLE Item_General_Trajectory
           ADD minTimestamp timestamptz;

           ALTER TABLE Item_General_Trajectory
           ADD maxTimestamp timestamptz;
        """
database._execute_query(query=query)

In [ ]:
### Update minTimestamp For Trajectories ###
query = """UPDATE Item_General_Trajectory
           SET minTimestamp = lower(trajCentroids::period)
           WHERE minTimestamp IS NULL;
        """
database._execute_query(query=query)

In [ ]:
### Update maxTimestamp For Trajectories ###
query = """UPDATE Item_General_Trajectory
           SET maxTimestamp = upper(trajCentroids::period)
           WHERE maxTimestamp IS NULL;
        """
database._execute_query(query=query)

In [ ]:
############ ROAD SEGMENT TYPES ############